image.png

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import os
import pandas as pd
import time  # 추가

2025-12-02 06:29:06.812315: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-02 06:29:09.006007: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-02 06:29:20.473314: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
# 사용자 설정
train_dir = '/workspace/user4/cropped/train'
val_dir = '/workspace/user4/cropped/val'
target_size = (224, 224)
batch_size = 32
seed = 42

# helper: 디렉터리를 스캔해서 파일경로와 라벨 DataFrame 생성
def make_dataframe_from_dir(base_dir, exclude_folders=('invalid',), positive_class='happy'):
    rows = []
    for class_name in sorted(os.listdir(base_dir)):
        class_path = os.path.join(base_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        if class_name in exclude_folders:
            continue
        label = 1 if class_name == positive_class else 0
        for fname in sorted(os.listdir(class_path)):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                rows.append({
                    'filepath': os.path.join(class_path, fname),
                    'label': label,
                    'orig_class': class_name
                })
    df = pd.DataFrame(rows)
    return df

# DataFrame 생성
train_df = make_dataframe_from_dir(train_dir, exclude_folders=('invalid',), positive_class='happy')
val_df = make_dataframe_from_dir(val_dir, exclude_folders=('invalid',), positive_class='happy')

print("Train samples:", len(train_df), "Val samples:", len(val_df))
print("Train class distribution:\n", train_df['label'].value_counts())
print("Val class distribution:\n", val_df['label'].value_counts())

# ImageDataGenerator
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# flow_from_dataframe
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='raw',
    shuffle=True,
    seed=seed
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='label',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='raw',
    shuffle=False,
    seed=seed
)

print("Generators created. Example batch shapes:")
x_batch, y_batch = next(train_generator)
print("x:", x_batch.shape, "y:", y_batch.shape, "y unique:", np.unique(y_batch))

# Base model 로드
base_model = EfficientNetB0(weights='imagenet', include_top=False,
                           input_shape=(target_size[0], target_size[1], 3))

# top 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# base freeze
for layer in base_model.layers:
    layer.trainable = False

# 컴파일
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

model.summary()

# step 계산
steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)

# 학습
print("\n--- 모델 학습 시작 ---")
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps,
)
print("--- 모델 학습 완료 ---\n")

# --- 검증 데이터셋 성능 평가 및 추론 속도 측정 ---
print("--- 검증 데이터셋 성능 평가 및 추론 속도 측정 시작 ---")

# 실제 레이블 추출
if hasattr(validation_generator, 'labels'):
    validation_labels = validation_generator.labels
elif hasattr(validation_generator, 'classes'):
    validation_labels = validation_generator.classes
else:
    validation_labels = val_df['label'].values

# 예측 수행 + 시간 측정
validation_generator.reset()
start_time = time.time()
pred_probabilities = model.predict(validation_generator, steps=validation_steps, verbose=1)
end_time = time.time()

elapsed_time = end_time - start_time
samples = len(val_df)
time_per_sample = elapsed_time / samples
print(f"\n총 추론 시간: {elapsed_time:.4f} 초")
print(f"샘플당 평균 추론 시간: {time_per_sample:.6f} 초")

# 확률 → 클래스 변환
pred_classes = (pred_probabilities > 0.5).astype(int).flatten()

# classification_report 출력
class_names = ['others', 'happy']
report = classification_report(validation_labels, pred_classes, target_names=class_names, digits=4)
print("\n--- Classification Report (Validation Set) ---\n")
print(report)

print("--- 검증 데이터셋 성능 평가 및 추론 속도 측정 완료 ---")


Train samples: 9550 Val samples: 1151
Train class distribution:
 label
1    5245
0    4305
Name: count, dtype: int64
Val class distribution:
 label
0    852
1    299
Name: count, dtype: int64
Found 9550 validated image filenames.
Found 1151 validated image filenames.
Generators created. Example batch shapes:
x: (32, 224, 224, 3) y: (32,) y unique: [0 1]


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ normalization_1[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,213,668 (16.07 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)


--- 모델 학습 시작 ---
Epoch 1/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 208s 641ms/step - accuracy: 0.8139 - auc: 0.8983 - loss: 0.4030 - val_accuracy: 0.7967 - val_auc: 0.9412 - val_loss: 0.4505
Epoch 2/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 176s 588ms/step - accuracy: 0.8733 - auc: 0.9452 - loss: 0.2985 - val_accuracy: 0.8801 - val_auc: 0.9536 - val_loss: 0.2873
Epoch 3/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 175s 587ms/step - accuracy: 0.8828 - auc: 0.9515 - loss: 0.2796 - val_accuracy: 0.8549 - val_auc: 0.9540 - val_loss: 0.3329
Epoch 4/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 188s 628ms/step - accuracy: 0.8963 - auc: 0.9574 - loss: 0.2624 - val_accuracy: 0.9062 - val_auc: 0.9595 - val_loss: 0.2340
Epoch 5/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 182s 608ms/step - accuracy: 0.8905 - auc: 0.9582 - loss: 0.2608 - val_accuracy: 0.9088 - val_auc: 0.9580 - val_loss: 0.2328
Epoch 6/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 185s 618ms/step - accuracy: 0.8939 - auc: 0.9609 - loss: 0.2519 - val_accuracy: 0.8349 - val_auc: 0.9605 - val_loss: 0.403

In [ ]:
# --- 추가된 코드 시작 ---

from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2  # 추가
from sklearn.metrics import accuracy_score


# 배치별 추론 시간 측정 (predict_on_batch 사용)
validation_generator.reset()
batch_times = []
batch_sizes = []
validation_steps = len(validation_generator)
for i in range(validation_steps):
    x_batch, y_batch = next(validation_generator)
    start = time.time()
    _ = model.predict_on_batch(x_batch)
    end = time.time()
    batch_times.append(end - start)
    batch_sizes.append(x_batch.shape[0])

per_sample_times = np.array(batch_times) / np.array(batch_sizes)
per_sample_times_ms = per_sample_times * 1000.0
mean_ms = per_sample_times_ms.mean()
std_ms = per_sample_times_ms.std()

# 파라미터 수 (M)
total_params = model.count_params()
params_m = total_params / 1e6

# FLOPs 계산 함수 및 시도
def get_flops(model, input_shape=(1, 224, 224, 3)):
    try:
        func = tf.function(lambda x: model(x))
        concrete = func.get_concrete_function(tf.TensorSpec(input_shape, tf.float32))
        frozen_func = convert_variables_to_constants_v2(concrete)
        graph_def = frozen_func.graph.as_graph_def()

        with tf.Graph().as_default():
            tf.import_graph_def(graph_def, name='')
            with tf.compat.v1.Session() as sess:
                run_meta = tf.compat.v1.RunMetadata()
                opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
                flops = tf.compat.v1.profiler.profile(sess.graph, run_meta=run_meta, cmd='op', options=opts)
                if flops is not None:
                    return flops.total_float_ops
    except Exception as e:
        print("FLOPs 계산 실패:", e)
    return None

flops_total = get_flops(model, input_shape=(1, target_size[0], target_size[1], 3))
flops_b = (flops_total / 1e9) if flops_total is not None else None

# 표 형식 출력 (첨부 이미지와 유사한 양식)
model_name = "EfficientNetB0"
input_size = target_size[0]
val_accuracy = accuracy_score(validation_labels, pred_classes)  # validation 정확도
accuracy_text = f"{val_accuracy*100:.2f}% (Val)"
speed_text = f"{mean_ms:.2f} ± {std_ms:.2f}"
params_text = f"{params_m:.1f}"
flops_text = f"{flops_b:.2f}" if flops_b is not None else "N/A"

print("="*60)
print("=== 모델 성능 요약 (YOLO 표 형식 참고) ===")
print("*참고: 분류 모델의 경우 mAP 대신 Validation 정확도를 사용합니다.")
print("| 모델       | 크기 (픽셀) | 정확도 (Validation) | 속도 CUDA (ms)   | 파라미터 (M) | FLOPs (B) |")
print("|------------|-------------|---------------------|------------------|--------------|-----------|")
print(f"| {model_name:<10} | {input_size:<11} | {accuracy_text:<19} | {speed_text:<16} | {params_text:<12} | {flops_text:<9} |")
print("="*60)

# 상세 classification report 출력 (기존에 없었다면 추가)
# print("\n--- Classification Report (Validation Set) ---")
# print(classification_report(validation_labels, pred_classes, digits=4))

# --- 추가된 코드 끝 ---


I0000 00:00:1764664539.038224  149657 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
I0000 00:00:1764664539.038470  149657 single_machine.cc:376] Starting new session
I0000 00:00:1764664539.045141  149657 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43710 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:53:00.0, compute capability: 8.6



=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
op: The nodes are operation kernel type, such as MatMul, Conv2D. Graph nodes belonging to the same type are aggregated together.
flops: Number of float operations. Note: Please read the implementation for th

I0000 00:00:1764664540.570060  149657 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43710 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:53:00.0, compute capability: 8.6


: 

In [ ]:

##### XXXXX
import torch
from tabulate import tabulate
import statistics 
import numpy as np # 파라미터/FLOPs 지표 출력을 위해 필요

# --- 1. 이전 측정 결과 불러오기 (이전 코드에서 계산된 변수 사용 가정) ---
# 다음 변수들이 이전 속도 측정 코드에서 정의되었다고 가정합니다:
# mean_syn (평균 추론 시간, ms)
# std_syn (표준 편차, ms)
# device (사용 장치, 'cuda' 또는 'cpu')

# 만약 이전에 측정한 mean_syn, std_syn이 없다면 임시 값을 설정하여 실행해야 합니다.
try:
    mean_syn = mean_syn
    std_syn = std_syn
    device = device
except NameError:
    print("경고: mean_syn, std_syn, device 변수가 정의되지 않아 임시 값으로 대체합니다. 속도 측정 코드를 먼저 실행하세요.")
    mean_syn = 5.0
    std_syn = 0.5
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# --- 2. 모델 (ImageNet 분류용)의 고정 지표 설정 ---
# 모델의 파라미터 및 FLOPs 값 (224x224 입력 기준)
PARAM_M = 11.69  # Million Parameters
FLOPS_B = 1.81   # Billion FLOPs

# 모델의 ImageNet Top-1 및 Top-5 정확도
TOP1_ACC = "69.7%"
TOP5_ACC = "89.0%"
ACCURACY_METRIC = f"{TOP1_ACC} (Top-1) / {TOP5_ACC} (Top-5)"

# --- 3. 표 데이터 구성 ---
# 표의 헤더 (CPU 속도 제외)
headers = [
    "모델", 
    "크기 (픽셀)", 
    "정확도 (ImageNet)", 
    f"속도 {device.type.upper()} (ms)", 
    "파라미터 (M)", 
    "FLOPs (B)"
]

# 측정 결과 포맷: 평균 ± 표준편차
speed_output = f"{mean_syn:.2f} ± {std_syn:.2f}"

# 표의 데이터 행
table_data = [
    ["EfficientNetB0", "224", ACCURACY_METRIC, speed_output, f"{PARAM_M:.1f}", f"{FLOPS_B:.1f}"]
]

# --- 4. 표 출력 ---
print("\n" + "=" * 85)
print("=== EfficientNetB0 성능 요약 (YOLO 표 형식 참고) ===")
print("  *참고: ImageNet Top-1/Top-5 정확도 사용")
print(tabulate(table_data, headers=headers, tablefmt="github"))
print("=" * 85)

경고: mean_syn, std_syn, device 변수가 정의되지 않아 임시 값으로 대체합니다. 속도 측정 코드를 먼저 실행하세요.

=== EfficientNetB0 성능 요약 (YOLO 표 형식 참고) ===
  *참고: ImageNet Top-1/Top-5 정확도 사용
| 모델           |   크기 (픽셀) | 정확도 (ImageNet)             | 속도 CUDA (ms)   |   파라미터 (M) |   FLOPs (B) |
|----------------|---------------|-------------------------------|------------------|----------------|-------------|
| EfficientNetB0 |           224 | 69.7% (Top-1) / 89.0% (Top-5) | 5.00 ± 0.50      |           11.7 |         1.8 |
